## 1.下载数据

In [1]:
import redis
import pickle
import time
import uuid
import pandas as pd
from io import StringIO
from typing import Any, Optional
from datetime import datetime, timedelta

class RemoteSender:
    def __init__(self, host='*', port=6379, password='*'):
        self.redis = redis.Redis(
            host=host, port=port, password=password,
            decode_responses=False
        )
        self.task_queue = 'function_calls'
        self.result_queue = 'function_results'
        self._test_connection()
        print(f"✅ 发送端pandas版本：{pd.__version__}")

    def _test_connection(self):
        try:
            self.redis.ping()
            print("✅ 发送端：Redis连接成功")
        except Exception as e:
            print(f"❌ 发送端：连接失败 - {e}")
            raise

    def call_remote_function(self, func_name: str, *args, **kwargs) -> Any:
        task_id = f"task_{uuid.uuid4().hex[:8]}"
        task = {
            'func_name': func_name,
            'args': args,
            'kwargs': kwargs,
            'task_id': task_id
        }
        self.redis.rpush(self.task_queue, pickle.dumps(task))
        print(f"📤 已调用远程函数：{func_name}（任务ID：{task_id}）")
        return self._get_result(task_id)

    def _get_result(self, task_id: str, timeout=300) -> Any:
        start_time = time.time()
        while time.time() - start_time < timeout:
            result_data = self.redis.blpop(self.result_queue, timeout=10)
            if not result_data:
                continue

            _, res_bytes = result_data
            result = pickle.loads(res_bytes)
            if result['task_id'] == task_id:
                if result['status'] == 'success':
                    return result['result']  # 返回CSV字符串
                else:
                    raise Exception(f"远程执行失败：{result['error']}")
            self.redis.rpush(self.result_queue, res_bytes)
        raise TimeoutError("任务超时")

    def save_to_csv(self, csv_str: Optional[str], filename: str) -> bool:
        """将CSV字符串保存为本地CSV文件（替代Parquet）"""
        if not csv_str:
            print("⚠️ 数据为空，不保存")
            return False
        try:
            # 从CSV字符串恢复DataFrame（兼容所有pandas版本）
            df = pd.read_csv(StringIO(csv_str))
            # 保存为CSV文件
            df.to_csv(filename, index=False)
            print(f"✅ 保存成功：{filename}（{len(df)}条记录）")
            return True
        except Exception as e:
            print(f"❌ 保存失败：{e}")
            return False

def generate_date_range(start_date_str: str, end_date_str: str) -> list:
    """生成从开始日期到结束日期的所有日期字符串（YYYYMMDD格式）"""
    dates = []
    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
        
        if start_date > end_date:
            raise ValueError("开始日期晚于结束日期")
            
        current_date = start_date
        while current_date <= end_date:
            dates.append(current_date.strftime('%Y%m%d'))
            current_date += timedelta(days=1)
    except Exception as e:
        print(f"日期处理错误：{e}")
    return dates

if __name__ == "__main__":

    # 从配置文件读取Redis连接信息
    with open('redis.conf', 'r') as f:
        for line in f:
            if line.startswith('host='):
                host = line.split('=')[1].strip()
            elif line.startswith('port='):
                port = int(line.split('=')[1].strip())
            elif line.startswith('password='):
                password = line.split('=')[1].strip()
    # 初始化Redis发送端
    sender = RemoteSender(host=host, port=port, password=password)
    
    # 定义日期范围：从20250516到20250923
    # 读取../data/stock_daily_price.parquet文件，获取最大的日期+1，是start_date
    df = pd.read_parquet('../data/stock_daily_marketcap.parquet')
    start_date = '20230101'#(df['date'].max() + timedelta(days=1)).strftime('%Y%m%d')
    # 获取当日日期-1，是end_date
    end_date = '20241231'#(datetime.today() - timedelta(days=1)).strftime('%Y%m%d')
    
    # 生成日期列表
    date_list = generate_date_range(start_date, end_date)
    print(f"=== 共需获取 {len(date_list)} 天的数据 ===")
    
    # 循环调用获取每日数据
    for i, date in enumerate(date_list, 1):
        print(f"\n=== 正在处理 {i}/{len(date_list)}：{date} ===")
        try:
            # 调用远程函数获取当日数据
            csv_data = sender.call_remote_function('fetch_daily_market_cap', date)
            # 保存为CSV文件，文件名包含日期
            sender.save_to_csv(csv_data, f'stock_daily_marketcap_{date}.csv')
            
            # 适当延迟，避免请求过于频繁
            time.sleep(0.1)
        except Exception as e:
            print(f"❌ {date} 处理失败：{e}")
            # 失败后也延迟一下，避免快速重试导致的问题
            time.sleep(2)
    
    print("\n=== 所有日期处理完成 ===")

✅ 发送端：Redis连接成功
✅ 发送端pandas版本：2.3.2
=== 共需获取 731 天的数据 ===

=== 正在处理 1/731：20230101 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_785533e9）
⚠️ 数据为空，不保存

=== 正在处理 2/731：20230102 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_ee4d1f62）
⚠️ 数据为空，不保存

=== 正在处理 3/731：20230103 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_89be52db）
✅ 保存成功：stock_daily_marketcap_20230103.csv（4905条记录）

=== 正在处理 4/731：20230104 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_88a6c3d4）
✅ 保存成功：stock_daily_marketcap_20230104.csv（4905条记录）

=== 正在处理 5/731：20230105 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_1ffa5f7f）
✅ 保存成功：stock_daily_marketcap_20230105.csv（4905条记录）

=== 正在处理 6/731：20230106 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_3bdbbe5e）
✅ 保存成功：stock_daily_marketcap_20230106.csv（4906条记录）

=== 正在处理 7/731：20230107 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_0ab28dac）
⚠️ 数据为空，不保存

=== 正在处理 8/731：20230108 ===
📤 已调用远程函数：fetch_daily_market_cap（任务ID：task_4e4f3980）
⚠️ 数据为空，不保存

=== 正在处理 9/731：20230109 ===
📤 已调用

## 将csv合并为一个parquet文件

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
将 /d:/workspace/xiaoyao/redis/ 目录下的所有 stock_***.csv 文件合并为一个 parquet 文件
确保与现有 /d:/workspace/xiaoyao/data/stock_daily_price.parquet 保持字段、压缩方式一致
"""

import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import glob


def merge_stock_csv_to_parquet(csv_dir, output_parquet_file):
    """
    合并指定目录下的所有 stock_***.csv 文件到单个 parquet 文件
    
    Args:
        csv_dir: CSV 文件所在目录
        output_parquet_file: 输出的 parquet 文件路径
    """
    print(f"📁 开始处理目录: {csv_dir}")
    
    # 获取所有 stock_***.csv 文件
    csv_pattern = os.path.join(csv_dir, "stock_daily_marketcap_*.csv")
    csv_files = glob.glob(csv_pattern)
    
    if not csv_files:
        print("❌ 未找到 stock_***.csv 文件")
        return False
    
    print(f"📊 找到 {len(csv_files)} 个 CSV 文件")
    
    # 按文件名排序（确保按日期顺序处理）
    csv_files.sort()
    
    # 读取并合并所有 CSV 文件
    all_dataframes = []
    total_records = 0
    
    for i, csv_file in enumerate(csv_files, 1):
        filename = os.path.basename(csv_file)
        print(f"正在处理 ({i}/{len(csv_files)}): {filename}")
        
        try:
            # 读取 CSV 文件
            df = pd.read_csv(csv_file)
            
            # 数据验证和清洗
            # 确保 date 列是 datetime 类型
            df['date'] = pd.to_datetime(df['date'])
            
            # 确保数值列的数据类型正确
            numeric_columns = ['capitalization','circulating_cap','market_cap','circulating_market_cap','turnover_ratio','pe_ratio','pe_ratio_lyr','pb_ratio','ps_ratio','pcf_ratio']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # 删除无效数据
            df = df.dropna(subset=['date', 'stock_code'])
            
            all_dataframes.append(df)
            total_records += len(df)
            print(f"  ✅ 成功读取 {len(df)} 条记录")
            
        except Exception as e:
            print(f"  ❌ 处理失败: {e}")
            continue
    
    if not all_dataframes:
        print("❌ 没有成功读取任何数据")
        return False
    
    print(f"\n📊 合并所有数据...")
    # 合并所有数据框
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    # 去重（按 date + stock_code）
    combined_df = combined_df.drop_duplicates(subset=['date', 'stock_code'])
    
    # 按日期和股票代码排序
    combined_df = combined_df.sort_values(['date', 'stock_code']).reset_index(drop=True)
    
    print(f"📈 总计 {len(combined_df)} 条记录（去重后）")
    
    # 确保输出目录存在
    output_dir = os.path.dirname(output_parquet_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 创建输出目录: {output_dir}")
    
    # 转换为 pyarrow Table
    table = pa.Table.from_pandas(combined_df)
    
    # 使用与目标文件相同的压缩方式 (snappy) 和格式写入 parquet
    try:
        pq.write_table(
            table, 
            output_parquet_file,
            compression='snappy',
            version='2.6',  # 使用较新的 parquet 版本
            use_dictionary=True,
            write_batch_size=64 * 1024 * 1024  # 64MB batch size for better performance
        )
        
        print(f"✅ 成功保存到: {output_parquet_file}")
        print(f"📊 文件大小: {os.path.getsize(output_parquet_file) / (1024*1024):.2f} MB")
        
        return True
        
    except Exception as e:
        print(f"❌ 保存 parquet 文件失败: {e}")
        return False


def main():
    """主函数"""
    # 设置路径
    csv_directory = "d:/workspace/xiaoyao/redis/"
    # 确保输出目录存在
    output_dir = os.path.join(csv_directory, 'parquet')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 创建输出目录: {output_dir}")

    output_file = os.path.join(output_dir, 'stock_daily_marketcap_to_merged.parquet')
    
    
    print("=" * 60)
    print("🚀 开始合并 stock_***.csv 文件到 parquet")
    print("=" * 60)
    
    # 执行合并
    success = merge_stock_csv_to_parquet(csv_directory, output_file)
    
    if success:
        print("\n🎉 合并完成！")
        
        # 验证结果
        try:
            print("\n📋 验证结果:")
            result_df = pd.read_parquet(output_file)
            print(f"   总行数: {len(result_df)}")
            print(f"   日期范围: {result_df['date'].min()} 到 {result_df['date'].max()}")
            print(f"   股票数量: {result_df['stock_code'].nunique()}")
            print(f"   列名: {list(result_df.columns)}")
            
        except Exception as e:
            print(f"⚠️  验证失败: {e}")
    
    else:
        print("\n❌ 合并失败！")
    
    print("=" * 60)


if __name__ == "__main__":
    main()

🚀 开始合并 stock_***.csv 文件到 parquet
📁 开始处理目录: d:/workspace/xiaoyao/redis/
📊 找到 483 个 CSV 文件
正在处理 (1/483): stock_daily_marketcap_20230103.csv
  ✅ 成功读取 4905 条记录
正在处理 (2/483): stock_daily_marketcap_20230104.csv
  ✅ 成功读取 4905 条记录
正在处理 (3/483): stock_daily_marketcap_20230105.csv
  ✅ 成功读取 4905 条记录
正在处理 (4/483): stock_daily_marketcap_20230106.csv
  ✅ 成功读取 4906 条记录
正在处理 (5/483): stock_daily_marketcap_20230109.csv
  ✅ 成功读取 4906 条记录
正在处理 (6/483): stock_daily_marketcap_20230110.csv
  ✅ 成功读取 4906 条记录
正在处理 (7/483): stock_daily_marketcap_20230111.csv
  ✅ 成功读取 4906 条记录
正在处理 (8/483): stock_daily_marketcap_20230112.csv
  ✅ 成功读取 4906 条记录
正在处理 (9/483): stock_daily_marketcap_20230113.csv
  ✅ 成功读取 4906 条记录
正在处理 (10/483): stock_daily_marketcap_20230116.csv
  ✅ 成功读取 4906 条记录
正在处理 (11/483): stock_daily_marketcap_20230117.csv
  ✅ 成功读取 4906 条记录
正在处理 (12/483): stock_daily_marketcap_20230118.csv
  ✅ 成功读取 4907 条记录
正在处理 (13/483): stock_daily_marketcap_20230119.csv
  ✅ 成功读取 4909 条记录
正在处理 (14/483): stock_daily_marketcap

## 将新生成的 stock_daily_auction_to_merged.parquet.parquet 与现有的 stock_daily_auction.parquet 合并

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
将新生成的 stock_daily_marketcap_to_merged.parquet.parquet 与现有的 stock_daily_marketcap.parquet 合并
"""

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os


def merge_parquet_files(existing_file, new_file, output_file):
    """
    合并两个 parquet 文件
    
    Args:
        existing_file: 现有的 parquet 文件路径
        new_file: 新的 parquet 文件路径  
        output_file: 输出的合并文件路径
    """
    print("📊 开始合并 parquet 文件...")
    
    try:
        # 读取现有数据
        print(f"📖 读取现有文件: {existing_file}")
        existing_df = pd.read_parquet(existing_file)
        print(f"   现有数据行数: {len(existing_df)}")
        
        # 读取新数据
        print(f"📖 读取新文件: {new_file}")
        new_df = pd.read_parquet(new_file)
        print(f"   新数据行数: {len(new_df)}")
        
        # 合并数据
        print("🔄 合并数据中...")
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        
        # 去重（按 date + stock_code）
        print("🧹 去重处理...")
        combined_df = combined_df.drop_duplicates(subset=['date', 'stock_code'])
        
        # 排序
        print("📅 按日期排序...")
        combined_df = combined_df.sort_values(['date', 'stock_code']).reset_index(drop=True)
        
        print(f"📈 合并后总行数: {len(combined_df)}")
        
        # 确保输出目录存在
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 转换为 pyarrow Table
        table = pa.Table.from_pandas(combined_df)
        
        # 写入 parquet（使用与源文件相同的格式）
        print(f"💾 保存合并结果: {output_file}")
        pq.write_table(
            table,
            output_file,
            compression='snappy',
            version='2.6',
            use_dictionary=True,
            write_batch_size=64 * 1024 * 1024
        )
        
        print(f"✅ 合并完成！文件大小: {os.path.getsize(output_file) / (1024*1024):.2f} MB")
        
        # 验证结果
        print("\n📋 验证结果:")
        result_df = pd.read_parquet(output_file)
        print(f"   最终行数: {len(result_df)}")
        print(f"   日期范围: {result_df['date'].min()} 到 {result_df['date'].max()}")
        print(f"   股票数量: {result_df['stock_code'].nunique()}")
        
        return True
        
    except Exception as e:
        print(f"❌ 合并失败: {e}")
        return False


def main():
    """主函数"""
    # 设置文件路径

    existing_file = "d:/workspace/xiaoyao/data/stock_daily_marketcap.parquet"
    new_file = "d:/workspace/xiaoyao/redis/parquet/stock_daily_marketcap_to_merged.parquet"
    output_file = "d:/workspace/xiaoyao/redis/parquet/stock_daily_marketcap.parquet"

    print("=" * 60)
    print("🚀 开始合并 parquet 文件")
    print("=" * 60)
    
    # 检查文件是否存在
    if not os.path.exists(existing_file):
        print(f"❌ 现有文件不存在: {existing_file}")
        return
    
    if not os.path.exists(new_file):
        print(f"❌ 新文件不存在: {new_file}")
        return
    
    # 执行合并
    success = merge_parquet_files(existing_file, new_file, output_file)
    
    if success:
        print("\n🎉 合并成功！")
    else:
        print("\n❌ 合并失败！")
    
    print("=" * 60)


if __name__ == "__main__":
    main()

🚀 开始合并 parquet 文件
📊 开始合并 parquet 文件...
📖 读取现有文件: d:/workspace/xiaoyao/data/stock_daily_marketcap.parquet
   现有数据行数: 998122
📖 读取新文件: d:/workspace/xiaoyao/redis/parquet/stock_daily_marketcap_to_merged.parquet
   新数据行数: 2442959
🔄 合并数据中...
🧹 去重处理...
📅 按日期排序...
📈 合并后总行数: 3441081
💾 保存合并结果: d:/workspace/xiaoyao/redis/parquet/stock_daily_marketcap.parquet
✅ 合并完成！文件大小: 155.89 MB

📋 验证结果:
   最终行数: 3441081
   日期范围: 2023-01-03 00:00:00 到 2025-10-23 00:00:00
   股票数量: 5284

🎉 合并成功！


## 删除已使用的csv

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
删除指定目录下的 stock_***.csv 文件
"""

import os
import glob
from pathlib import Path


def delete_stock_csv_files(target_directory, pattern="stock_*.csv"):
    #删除满足模式的所有文件
    files = glob.glob(os.path.join(target_directory, pattern))
    for file in files:
        try:
            os.remove(file)
            print(f"已删除：{file}")
        except Exception as e:
            print(f"删除 {file} 失败：{e}")

if __name__ == "__main__":
    delete_stock_csv_files(r'D:\workspace\xiaoyao\redis','stock_daily_marketcap_*.csv')
    print("\n" + "=" * 60)

已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230103.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230104.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230105.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230106.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230109.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230110.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230111.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230112.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230113.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230116.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230117.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230118.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230119.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230120.csv
已删除：D:\workspace\xiaoyao\redis\stock_daily_marketcap_20230130.csv
已删除：D:\wor

## 将新的parquet文件移动到data目录覆盖原文件

In [5]:
# 将子目录下的某个parquet文件移动到指定目录
import os
import shutil

# 定义源目录和目标目录
source_dir = "D:\\workspace\\xiaoyao\\redis\\parquet"
target_dir = "D:\\workspace\\xiaoyao\\data"

# 确保目标目录存在
os.makedirs(target_dir, exist_ok=True)

# 定义要移动的文件
file_to_move = "stock_daily_marketcap.parquet"

# 构建源文件的完整路径
source_file_path = os.path.join(source_dir, file_to_move)

# 构建目标文件的完整路径
target_file_path = os.path.join(target_dir, file_to_move)

# 检查源文件是否存在
if os.path.exists(source_file_path):
    # 移动文件
    shutil.move(source_file_path, target_file_path)
    print(f"Moved: {file_to_move} to {target_dir}")
else:
    print(f"File not found: {file_to_move}")

# 删除指定的parquet文件
file_to_delete = os.path.join(source_dir, 'stock_daily_marketcap_to_merged.parquet')
if os.path.exists(file_to_delete):
    os.remove(file_to_delete)
    print(f"Deleted: {file_to_move} from {source_dir}")
else:
    print(f"File not found: {file_to_move} in {source_dir}")

Moved: stock_daily_marketcap.parquet to D:\workspace\xiaoyao\data
Deleted: stock_daily_marketcap.parquet from D:\workspace\xiaoyao\redis\parquet
